<a href="https://colab.research.google.com/github/ykitaguchi77/grav_bootcamp/blob/master/YOLOv5_dataset_to_bootcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**YOLOv5用データセット --> Bootcamp用セット作成**

In [75]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import pandas as pd
import glob
import os


for dataset in ["train", "valid"]:
    dataset_path = f"/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/{dataset}" #train or valid 

    txt_files = []  # txtファイルのパスを格納するリスト
    image_paths, name_list, label_list = [],  [], []

    # labelフォルダ内のテキストの中の１番目の数字がラベル
    for file_name in os.listdir(f"{dataset_path}/labels"):
        if file_name.endswith('.txt'):
            file_path = f"{dataset_path}/labels/{file_name}"
            name = os.path.basename(file_path).split(".")[0] 
            name_list.append(name.split("-")[0]) #患者IDを抜き出す
            image_paths.append(f"{dataset_path}/images/{name}.JPG")

            with open(file_path, 'r') as f:
                for line in f:
                    label = line.split()[0]
                    label_list.append(label)

    # データフレームを作成する
    df = pd.DataFrame({'image_path': image_paths, 'name': name_list, 'label': label_list})
    print(f"{dataset}: {len(df)}")

    # データフレームを表示する
    df.to_csv(f'/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/{dataset}_list.csv', index=False)




train: 2649
valid: 664


#**Valid_dataset --> hum_evalへ**

validデータセットから、なるべく患者が被らないように66×3セットの人力評価用データセットを作成する

In [109]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/valid_list.csv')
# df = df.sort_values('name') #nameで並べ替え
# df = df.sort_values('label') #labelで並べ替え
# df = df.reset_index() #index振り直し
# df = df.drop('index', axis=1) #indexの列を削除
# df
#df

In [118]:
import pandas as pd

def extract_dataset(df):
    # nameが重複していないdfを作る（かぶっているものをランダムにdropした）
    rows_no_name_duplicate = df.groupby("name").apply(lambda x: x.sample(1)).reset_index(drop=True)

    # "label"が0の行を取得
    label_0_rows = rows_no_name_duplicate[rows_no_name_duplicate["label"]==0]

    # "label"が1の行を取得
    label_1_rows = rows_no_name_duplicate[rows_no_name_duplicate["label"]==1]

    # ランダムに33個選択
    random_label_0_rows = label_0_rows.sample(n=33, random_state=1)

    # ランダムに33個選択
    random_label_1_rows = label_1_rows.sample(n=33, random_state=1)

    # 上記で抜粋したデータを消去したリストを返す
    df_concat = pd.concat([random_label_0_rows, random_label_1_rows])
    df_remain = df[~df.isin(df_concat)].dropna()

    return random_label_0_rows, random_label_1_rows, df_remain


# データセットからIDの被らない33例ずつ（grav, cont)を抜粋
step0_cont, step0_grav, df = extract_dataset(df)
step1_cont, step1_grav, df = extract_dataset(df)
step2_cont, step2_grav, df = extract_dataset(df)

#　各ステップのデータセットをシャッフルする
phase1 = pd.concat([step0_cont, step0_grav]).sample(frac=1, random_state=2).reset_index(drop=True)
phase2 = pd.concat([step1_cont, step1_grav]).sample(frac=1, random_state=3).reset_index(drop=True)
phase3 = pd.concat([step2_cont, step2_grav]).sample(frac=1, random_state=4).reset_index(drop=True)

In [117]:
step1


,image_path,name,label
0,/content/drive/MyDrive/Deep_learning/GO_extend...,6407,0.0
1,/content/drive/MyDrive/Deep_learning/GO_extend...,4915,0.0
2,/content/drive/MyDrive/Deep_learning/GO_extend...,489,1.0
3,/content/drive/MyDrive/Deep_learning/GO_extend...,2741,0.0
4,/content/drive/MyDrive/Deep_learning/GO_extend...,7283,1.0
...,...,...,...
61,/content/drive/MyDrive/Deep_learning/GO_extend...,6679,0.0
62,/content/drive/MyDrive/Deep_learning/GO_extend...,9747,0.0
63,/content/drive/MyDrive/Deep_learning/GO_extend...,7104,1.0
64,/content/drive/MyDrive/Deep_learning/GO_extend...,5937,0.0


#**評価用CSVを作成**

In [132]:
def make_idxs(phase):
    idxs = [f"phase{phase}_{i:02d}" for i in range(66)]
    return idxs


idxs = make_idxs(1) + make_idxs(2) + make_idxs(3)
labels = phase1["label"].tolist() + phase2["label"].tolist() + phase3["label"].tolist()
paths = phase1["image_path"].tolist() + phase2["image_path"].tolist() + phase3["image_path"].tolist()

198
198
198


In [127]:
columns = ["idx", "label", "path"]
participants = ["ishihara", "ohagi", "kinoshita", "takamine", "hayashi", "fukuyama", "makino", "motomura", "iwasaki", "komatsu", "koizumi", "kitaguchi", "YOLOv5"] 
columns.extend(participants)

df = pd.DataFrame(index=[], columns=columns)
df["idx"] = idxs
df["label"] = labels
df["path"] = paths

In [128]:
df

,idx,label,path,ishihara,ohagi,kinoshita,takamine,hayashi,fukuyama,makino,motomura,iwasaki,komatsu,koizumi,kitaguchi,YOLOv5
0,phase1_00,1.0,/content/drive/MyDrive/Deep_learning/GO_extend...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,phase1_01,0.0,/content/drive/MyDrive/Deep_learning/GO_extend...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,phase1_02,1.0,/content/drive/MyDrive/Deep_learning/GO_extend...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,phase1_03,1.0,/content/drive/MyDrive/Deep_learning/GO_extend...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,phase1_04,2.0,/content/drive/MyDrive/Deep_learning/GO_extend...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,phase3_61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,phase3_62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,phase3_63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,phase3_64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
labels

0     1.0
1     0.0
2     1.0
3     1.0
4     2.0
     ... 
61    1.0
62    0.0
63    2.0
64    1.0
65    2.0
Name: label, Length: 66, dtype: float64